<a href="https://colab.research.google.com/github/cahide/ibm-skillsbuild-preventive-healthcare-chatbot/blob/main/IBM_SkillsBuild_HealthCare_Chatbot_Data_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies (run once in Colab)
!pip install ibm-watsonx-ai beautifulsoup4 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.14.3-py3-none-any.whl size=77232 sha256=aaf194c7405b89d08547151fa83ddc178a7563c11a3b92790b4c4363e9807936
  Stored in directory: /root/.cache/pip/wheels/cc/2f/6f/125918ad46d280d3bea58edf99f0757888ef6e7999db4b73b7
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.14.3-py3-none-any.whl size=662101 sha256=ee8f56a79010efdad566f2c352f13d1a3613d2722bc3f94e8332fffc79d9be18
  Stored in directory: /root/.cache/pip/wheels/f1/53/13/7c8fdeebdb847995d8ef349b4f695c595d8d31b30ae2a07ea2
  Created whe

In [17]:
from google.colab import userdata

api_key = userdata.get('WATSONX_API_KEY')
project_id = userdata.get('WATSONX_PROJECT_ID')

In [20]:
import json
import requests
from bs4 import BeautifulSoup
from getpass import getpass
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
import re

# -----------------------
#  Watsonx credentials
# -----------------------
instance_url = "https://us-south.ml.cloud.ibm.com"

credentials = {"url": instance_url, "apikey": api_key}

# -----------------------
# Granite model setup
# -----------------------
gen_params = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 400,
    "temperature": 0.7
}

model = ModelInference(
    model_id="ibm/granite-3-8b-instruct",
    credentials=credentials,
    project_id=project_id,
    params=gen_params,
    verify=False
)

# -----------------------
# Scraping public sources
# -----------------------
def scrape_text(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    paragraphs = soup.find_all('p')
    # return paragraphs as list of strings
    return [p.get_text().strip() for p in paragraphs if p.get_text().strip()]

# -----------------------
# Health paragraph filter
# -----------------------
health_keywords = ["health", "exercise", "diet", "nutrition", "disease",
                   "prevention", "well-being", "diabetes", "fitness", "cardiovascular", "obesity"]

def is_health_paragraph(text):
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in health_keywords)

# -----------------------
# Generate QA pairs (text parsing)
# -----------------------
qa_list = []

def generate_qa_text(paragraph, source, topic):
    prompt = (
        "Create 3-5 question-answer pairs from the following paragraph. "
        "Output in plain text only with the format:\n"
        "Q: <question>\nA: <answer>\n\n"
        "Do NOT output JSON, numbering, or extra text.\n\n"
        f"{paragraph}"
    )
    response = model.generate_text(prompt=prompt)

    # Regex to extract Q/A pairs
    qa_matches = re.findall(r"Q:\s*(.+?)\nA:\s*(.+?)(?:\n\n|$)", response, flags=re.DOTALL)

    for q, a in qa_matches:
        difficulty = "Basic" if len(a.split()) <= 20 else "Intermediate" if len(a.split()) <= 40 else "Advanced"
        qa_list.append({
            "question": q.strip(),
            "answer": a.strip(),
            "source": source,
            "topic": topic
        })
# -----------------------
# Load URLs from JSON and process
# -----------------------
with open("scrape_url.json") as f:
    urls = json.load(f)


for entry in urls:
    source = entry.get("source", "Health")
    topic = entry.get("topic", "Health")
    url = entry["url"]
    paragraphs = scrape_text(url)
    for para in paragraphs:
        if is_health_paragraph(para):
            generate_qa_text(para, source, topic)

# -----------------------
# Save JSON
# -----------------------
with open("preventive_health_faq.json", "w") as f:
    json.dump(qa_list, f, indent=2)

print(f"Saved {len(qa_list)} QA pairs to preventive_health_faq.json")

# Optional: download in Colab
from google.colab import files
files.download("/content/preventive_health_faq.json")


Saved 1620 QA pairs to preventive_health_faq.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>